# 查看Pipeline支持的任务类型

In [ ]:
## 学术加速

In [6]:
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [1]:
from transformers.pipelines import SUPPORTED_TASKS

In [ ]:
print(SUPPORTED_TASKS.items())

dict_items([('audio-classification', {'impl': <class 'transformers.pipelines.audio_classification.AudioClassificationPipeline'>, 'tf': (), 'pt': (<class 'transformers.models.auto.modeling_auto.AutoModelForAudioClassification'>,), 'default': {'model': {'pt': ('superb/wav2vec2-base-superb-ks', '372e048')}}, 'type': 'audio'}), ('automatic-speech-recognition', {'impl': <class 'transformers.pipelines.automatic_speech_recognition.AutomaticSpeechRecognitionPipeline'>, 'tf': (), 'pt': (<class 'transformers.models.auto.modeling_auto.AutoModelForCTC'>, <class 'transformers.models.auto.modeling_auto.AutoModelForSpeechSeq2Seq'>), 'default': {'model': {'pt': ('facebook/wav2vec2-base-960h', '55bb623')}}, 'type': 'multimodal'}), ('text-to-audio', {'impl': <class 'transformers.pipelines.text_to_audio.TextToAudioPipeline'>, 'tf': (), 'pt': (<class 'transformers.models.auto.modeling_auto.AutoModelForTextToWaveform'>, <class 'transformers.models.auto.modeling_auto.AutoModelForTextToSpectrogram'>), 'defau

In [2]:
for k, v in SUPPORTED_TASKS.items():
    print(k, v)

audio-classification {'impl': <class 'transformers.pipelines.audio_classification.AudioClassificationPipeline'>, 'tf': (), 'pt': (<class 'transformers.models.auto.modeling_auto.AutoModelForAudioClassification'>,), 'default': {'model': {'pt': ('superb/wav2vec2-base-superb-ks', '372e048')}}, 'type': 'audio'}
automatic-speech-recognition {'impl': <class 'transformers.pipelines.automatic_speech_recognition.AutomaticSpeechRecognitionPipeline'>, 'tf': (), 'pt': (<class 'transformers.models.auto.modeling_auto.AutoModelForCTC'>, <class 'transformers.models.auto.modeling_auto.AutoModelForSpeechSeq2Seq'>), 'default': {'model': {'pt': ('facebook/wav2vec2-base-960h', '55bb623')}}, 'type': 'multimodal'}
text-to-audio {'impl': <class 'transformers.pipelines.text_to_audio.TextToAudioPipeline'>, 'tf': (), 'pt': (<class 'transformers.models.auto.modeling_auto.AutoModelForTextToWaveform'>, <class 'transformers.models.auto.modeling_auto.AutoModelForTextToSpectrogram'>), 'default': {'model': {'pt': ('suno

# Pipeline的创建与使用方式

In [7]:
from transformers import pipeline

## 根据任务类型直接创建Pipeline, 默认都是英文的模型

In [ ]:
pipe = pipeline("text-classification")

In [ ]:
pipe(["very good!", "vary bad!"])

## 指定任务类型，再指定模型，创建基于指定模型的Pipeline

In [9]:
# https://huggingface.co/models
pipe = pipeline("text-classification", model="uer/roberta-base-finetuned-dianping-chinese")

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/409M [00:00<?, ?B/s]

/root/miniconda3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/295 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [10]:
pipe("我觉得不太行！")

[{'label': 'negative (stars 1, 2 and 3)', 'score': 0.9735506772994995}]

## 预先加载模型，再创建Pipeline

In [13]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# 这种方式，必须同时指定model和tokenizer
model = AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")
tokenizer = AutoTokenizer.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer)

In [14]:
pipe("今天天气很晴朗，处处好风光！")

[{'label': 'positive (stars 4 and 5)', 'score': 0.9193938374519348}]

In [16]:
pipe.model.device

device(type='cpu')

In [17]:
import torch
import time
times = []
for i in range(100):
    torch.cuda.synchronize()
    start = time.time()
    pipe("今天天气很晴朗，处处好风光！")
    torch.cuda.synchronize()
    end = time.time()
    times.append(end - start)
print(sum(times) / 100)

0.015748867988586424


## 使用GPU进行推理

In [18]:
pipe = pipeline("text-classification", model="uer/roberta-base-finetuned-dianping-chinese", device=0)

In [20]:
pipe.model.device

device(type='cuda', index=0)

In [21]:
import torch
import time
times = []
for i in range(100):
    torch.cuda.synchronize()
    start = time.time()
    pipe("我觉得不太行！")
    torch.cuda.synchronize()
    end = time.time()
    times.append(end - start)
print(sum(times) / 100)

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


0.0058620524406433104


## 确定Pipeline参数

In [22]:
qa_pipe = pipeline("question-answering", model="uer/roberta-base-chinese-extractive-qa")

config.json:   0%|          | 0.00/452 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/407M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/216 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [23]:
qa_pipe

In [ ]:
QuestionAnsweringPipeline 

In [26]:
qa_pipe(question="云南的省会是哪里？", context="云南的省会是昆明", max_answer_len=2)

{'score': 0.876051127910614, 'start': 6, 'end': 8, 'answer': '昆明'}

# Pipeline背后的实现

In [27]:
from transformers import *
import torch

/root/miniconda3/lib/python3.10/site-packages/transformers/deepspeed.py:23: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(


In [29]:
tokenizer = AutoTokenizer.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")
model = AutoModelForSequenceClassification.from_pretrained("uer/roberta-base-finetuned-dianping-chinese")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--uer--roberta-base-finetuned-dianping-chinese/snapshots/25faf1874b21e76db31ea9c396ccf2a0322e0071/config.json
Model config BertConfig {
  "_name_or_path": "uer/roberta-base-finetuned-dianping-chinese",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "negative (stars 1, 2 and 3)",
    "1": "positive (stars 4 and 5)"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "negative (stars 1, 2 and 3)": 0,
    "positive (stars 4 and 5)": 1
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.40.0",
  "type_vocab_si

In [30]:
input_text = "今天天气好晴朗，处处好风光！"
#调用分词器对输入文本进行处理，并返回PyTorch张量（tensor）的形式。
#这里的inputs变量将包含分词器处理后的结果，它是一个字典，包含分词器生成的编码张量、注意力掩码（attention mask）等信息
inputs = tokenizer(input_text, return_tensors="pt")
inputs

{'input_ids': tensor([[ 101,  791, 1921, 1921, 3698, 1962, 3252, 3306, 8024, 1905, 1905, 1962,
         7599, 1045, 8013,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [31]:
#这段代码的意思是将 **inputs（即经过分词器处理的输入文本）作为参数传递给模型进行推理。
res = model(**inputs)
res

SequenceClassifierOutput(loss=None, logits=tensor([[-1.0786,  0.9897]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [32]:
##获取了模型的输出logits。Logits是模型在每个可能的类别上的得分或概率，还没有经过概率归一化处理。
logits = res.logits
## 使用PyTorch中的torch.softmax函数对logits进行softmax操作。softmax函数将每个logit值转换为在[0, 1]范围内的概率值，使得所有类别的概率之和为1
## dim=-1表示在最后一个维度上进行softmax操作。在情感分类任务中，通常最后一个维度是类别维度
logits = torch.softmax(logits, dim=-1)
logits

tensor([[0.1122, 0.8878]], grad_fn=<SoftmaxBackward0>)

In [33]:
pred = torch.argmax(logits).item()
pred

1

In [34]:
model.config.id2label

{0: 'negative (stars 1, 2 and 3)', 1: 'positive (stars 4 and 5)'}

In [35]:
result = model.config.id2label.get(pred)
result

'positive (stars 4 and 5)'